# Prepare environment

In [1]:
!pip install git+https://github.com/katarinagresova/ensembl_scraper.git@6d3bba8e6be7f5ead58a3bbaed6a4e8cd35e62fd

  Cloning https://github.com/katarinagresova/ensembl_scraper.git (to revision 6d3bba8e6be7f5ead58a3bbaed6a4e8cd35e62fd) to /tmp/pip-req-build-fz97hoif
  Running command git clone --filter=blob:none -q https://github.com/katarinagresova/ensembl_scraper.git /tmp/pip-req-build-fz97hoif
  Running command git rev-parse -q --verify 'sha^6d3bba8e6be7f5ead58a3bbaed6a4e8cd35e62fd'
  Running command git fetch -q https://github.com/katarinagresova/ensembl_scraper.git 6d3bba8e6be7f5ead58a3bbaed6a4e8cd35e62fd
  Resolved https://github.com/katarinagresova/ensembl_scraper.git to commit 6d3bba8e6be7f5ead58a3bbaed6a4e8cd35e62fd
  Preparing metadata (setup.py) ... done
You should consider upgrading via the '/home/katarina/git/genomic_benchmarks/venv/bin/python -m pip install --upgrade pip' command.


# Create config file

In [5]:
import yaml

config = {
    "root_dir": "../../datasets/",
    "organisms": {
        "homo_sapiens": {
            "regulatory_feature"
        }
    }
}

user_config = 'user_config.yaml'
with open(user_config, 'w') as handle:
  yaml.dump(config, handle)

# Prepare directories

In [6]:
from pathlib import Path

BASE_FILE_PATH = Path("../../datasets/human_ensembl_regulatory/")

# copied from https://stackoverflow.com/a/57892171
def rm_tree(pth: Path):
    for child in pth.iterdir():
        if child.is_file():
            child.unlink()
        else:
            rm_tree(child)
    pth.rmdir()

if BASE_FILE_PATH.exists():
    rm_tree(BASE_FILE_PATH)

# Run tool

In [7]:
!python -m scraper.ensembl_scraper -c user_config.yaml

Processing feature files:   0%|                           | 0/1 [00:00<?, ?it/s]INFO:root:download_file(): Going to download file from path ftp://ftp.ensembl.org/pub/release-100/mysql/regulation_mart_100/hsapiens_regulatory_feature__regulatory_feature__main.txt.gz
INFO:root:download_file(): File downloaded to path ../../datasets//tmp//homo_sapiens_regulatory_feature.txt.gz.
INFO:root:parse_feature_file(): Going to parse file ../../datasets//tmp//homo_sapiens_regulatory_feature.txt.gz
INFO:root:parse_feature_file(): Done parsing file ../../datasets//tmp//homo_sapiens_regulatory_feature.txt.gz


Processing feature types:   0%|                           | 0/6 [00:00<?, ?it/s]INFO:root:find_sequences(): Going to find sequences based on genomic loci.
INFO:root:download_2bit_file(): Going to download 2bit file hg38
INFO:root:download_2bit_file(): File for hg38 downloaded to path ../../datasets//tmp/hg38.2bit.
INFO:root:find_sequences(): Done finding sequences.
INFO:root:remove_low_quality():

# Reformating

In [9]:
!mkdir -p ../../datasets/human_ensembl_regulatory/train
!mkdir -p ../../datasets/human_ensembl_regulatory/test

In [ ]:
!mv ../../datasets/homo_sapiens_regulatory_feature_open_chromatin_region/train/positive.csv ../../datasets/human_ensembl_regulatory/train/ocr.csv
!mv ../../datasets/homo_sapiens_regulatory_feature_open_chromatin_region/test/positive.csv ../../datasets/human_ensembl_regulatory/test/ocr.csv

!mv ../../datasets/homo_sapiens_regulatory_feature_promoter/train/positive.csv ../../datasets/human_ensembl_regulatory/train/promoter.csv
!mv ../../datasets/homo_sapiens_regulatory_feature_promoter/test/positive.csv ../../datasets/human_ensembl_regulatory/test/promoter.csv

!mv ../../datasets/homo_sapiens_regulatory_feature_enhancer/train/positive.csv ../../datasets/human_ensembl_regulatory/train/enhancer.csv
!mv ../../datasets/homo_sapiens_regulatory_feature_enhancer/test/positive.csv ../../datasets/human_ensembl_regulatory/test/enhancer.csv

In [48]:
def chop_sequnces(file_path, max_len):

    df = pd.read_csv(file_path)
    df_array = df.values

    new_df_array = []
    index = 0
    for row in df_array:
        splits = ((row[3] - row[2]) // max_len) + 1

        if splits == 1:
            new_df_array.append([index, row[1], row[2], row[3], row[4]])
            index += 1

        elif splits == 2:
            length = (row[3] - row[2]) // 2
            new_df_array.append([
                index,
                row[1],
                row[2],
                row[2] + length,
                row[4]           
            ])
            index += 1
            new_df_array.append([
                index,
                row[1],
                row[2] + length + 1,
                row[3],
                row[4]           
            ])
            index += 1
        else:
            length = (row[3] - row[2]) // splits
            new_df_array.append([
                index,
                row[1],
                row[2],
                row[2] + length,
                row[4]           
            ])
            index += 1
            for i in range(1, splits - 1):
                new_df_array.append([
                    index,
                    row[1],
                    row[2] + i*length + 1,
                    row[2] + (i + 1)*length,
                    row[4]           
                ])
                index += 1

            new_df_array.append([
                index,
                row[1],
                row[2] + (splits - 1)*length + 1,
                row[3],
                row[4]           
            ])
            index += 1
    new_df = pd.DataFrame(new_df_array, columns=df.columns)
    new_df.to_csv(file_path, index=False)

In [49]:
chop_sequnces("../../datasets/human_ensembl_regulatory/train/promoter.csv", 700)
chop_sequnces("../../datasets/human_ensembl_regulatory/test/promoter.csv", 700)

In [51]:
!find ../../datasets/human_ensembl_regulatory/ -type f -name "*.csv" -exec gzip {} \;

In [13]:
!mv ../../datasets/homo_sapiens_regulatory_feature_enhancer/metadata.yaml ../../datasets/human_ensembl_regulatory/metadata.yaml

In [14]:
with open("../../datasets/human_ensembl_regulatory/metadata.yaml", "r") as stream:
    try:
        config = yaml.safe_load(stream)
    except yaml.YAMLError as exc:
        print(exc)

config

{'classes': {'negative': {'type': 'fa.gz',
   'url': 'ftp://ftp.ensembl.org/pub/release-100/fasta/homo_sapiens/dna/Homo_sapiens.GRCh38.dna.toplevel.fa.gz'},
  'positive': {'type': 'fa.gz',
   'url': 'ftp://ftp.ensembl.org/pub/release-100/fasta/homo_sapiens/dna/Homo_sapiens.GRCh38.dna.toplevel.fa.gz'}},
 'version': 0}

In [16]:
new_config = {
    'classes' : {
        'ocr': {
            'type': config['classes']['positive']['type'],
            'url': config['classes']['positive']['url'],
            'extra_processing': 'ENSEMBL_HUMAN_GENOME'
        },
        'promoter': {
            'type': config['classes']['positive']['type'],
            'url': config['classes']['positive']['url'],
            'extra_processing': 'ENSEMBL_HUMAN_GENOME'
        },
        'enhancer': {
            'type': config['classes']['positive']['type'],
            'url': config['classes']['positive']['url'],
            'extra_processing': 'ENSEMBL_HUMAN_GENOME'
        }
    },
    'version': config['version']
}
new_config 

{'classes': {'ocr': {'type': 'fa.gz',
   'url': 'ftp://ftp.ensembl.org/pub/release-100/fasta/homo_sapiens/dna/Homo_sapiens.GRCh38.dna.toplevel.fa.gz',
   'extra_processing': 'ENSEMBL_HUMAN_GENOME'},
  'promoter': {'type': 'fa.gz',
   'url': 'ftp://ftp.ensembl.org/pub/release-100/fasta/homo_sapiens/dna/Homo_sapiens.GRCh38.dna.toplevel.fa.gz',
   'extra_processing': 'ENSEMBL_HUMAN_GENOME'},
  'enhancer': {'type': 'fa.gz',
   'url': 'ftp://ftp.ensembl.org/pub/release-100/fasta/homo_sapiens/dna/Homo_sapiens.GRCh38.dna.toplevel.fa.gz',
   'extra_processing': 'ENSEMBL_HUMAN_GENOME'}},
 'version': 0}

In [18]:
with open("../../datasets/human_ensembl_regulatory/metadata.yaml", 'w') as handle:
  yaml.dump(new_config, handle)

# Cleaning

In [4]:
!rm user_config.yaml
!rm -rf ../../datasets/tmp/

!rm -rf ../../datasets/homo_sapiens_regulatory_feature_CTCF_binding_site
!rm -rf ../../datasets/homo_sapiens_regulatory_feature_enhancer
!rm -rf ../../datasets/homo_sapiens_regulatory_feature_promoter
!rm -rf ../../datasets/homo_sapiens_regulatory_feature_promoter_flanking_region
!rm -rf ../../datasets/homo_sapiens_regulatory_feature_TF_binding_site
!rm -rf ../../datasets/homo_sapiens_regulatory_feature_open_chromatin_region


# Testing

In [52]:
from genomic_benchmarks.loc2seq import download_dataset

download_dataset("human_ensembl_regulatory", local_repo=True)

Reference /home/katarina/.genomic_benchmarks/fasta/Homo_sapiens.GRCh38.dna.toplevel.fa.gz already exists. Skipping.


100%|██████████| 24/24 [00:32<00:00,  1.36s/it]


PosixPath('/home/katarina/.genomic_benchmarks/human_ensembl_regulatory')

In [53]:
from genomic_benchmarks.data_check import info

info("human_ensembl_regulatory", 0, local_repo=True)

Dataset `human_ensembl_regulatory` has 3 classes: enhancer, ocr, promoter.

The length of genomic intervals ranges from 71 to 802, with average 429.91753643694585 and median 401.0.

Totally 289061 sequences have been found, 231348 for training and 57713 for testing.


,train,test
enhancer,85512,21378
ocr,69902,17476
promoter,75934,18859
